In [6]:
import pickle

from .pre_process import * 
from .ocr_process import *
from .post_process1 import *
from .post_process2 import *


SystemError: Parent module '' not loaded, cannot perform relative import

In [4]:
dic_path = Path(__file__).ancestor(2).child('dictionary')
w2_path = dic_path.child('w2_dictionary.pk1')


def autocomplete(url=None, img=None):
    with open(w2_path, 'rb') as f:
        w2_dic = pickle.load(f)
    img_list = pre_process(url=url, img=img, show=False)
    page = ocr_process(img_list)
    result = post_process1(page=page, w2_dic=w2_dic)
    rdbs = post_process2(result=result)
    return rdbs

NameError: name '__file__' is not defined